Scraping PV data
----------
In this file, I am scraping the link http://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/ from the data it has.

upon inspection, it contains nested zips that contains more dates to the same zips. (example: file `PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223` outputs one CSV when exported manually but outputs 112 zips when extracted programmatically (command line or python)

The steps are as follows:
- I get the zip urls from the website and store them in a list
- iterate over the list and download them into a folder 
-[ TODO ] iterate over the zips in `unzipped_files` and export them to CSVs
- read the CSVs in pandas and cocat into the same dataframe

# Get ZIP URLS

In [ ]:
import requests
from bs4 import BeautifulSoup
# from urllib.parse import urlparse


# The URL of the page you want to scrape
url = "http://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/"
base_url= "http://nemweb.com.au"
# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    zip_urls=[]
    # Find all <a> tags
    links = soup.find_all('a')
    
    
    # Filter out links and print them
    for link in links:
        href = link.get('href')
        print(base_url)
        if href and href.endswith('.zip'):  # Check if the link ends with .zip
            full_url = base_url + href if not href.startswith('http') else href
#             print(href)
#             print(full_url)
            zip_urls.append(full_url)
else:
    print(f"Failed to retrieve content from {url}, status code: {response.status_code}")


# Download the zips

In [ ]:
import requests
import os

# Define the folder name where files will be saved
folder_name = 'zips'

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for file_url in zip_urls:
    print(file_url)
    response = requests.get(file_url)
    filename = file_url.split('/')[-1]  # Extracts the filename

    # Modify the path to save the file in the new folder
    file_path = os.path.join(folder_name, filename)

    # Save the file in the specified folder
    with open(file_path, 'wb') as file:
        file.write(response.content)

    print(f"Downloaded {filename} into {folder_name}")


# Unzip 1

In [ ]:
import os
import zipfile

# Define the source and destination folder paths
source_folder = 'zips'
destination_folder = 'unzipped_files'

# Check if the destination folder exists, if not, create it
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# List all the files in the source folder
for filename in os.listdir(source_folder):
    # Construct the full file path
    file_path = os.path.join(source_folder, filename)
    
    # Check if the file is a ZIP file
    if zipfile.is_zipfile(file_path):
        print(f"Unzipping {filename}")
        
        # Open the ZIP file
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Extract all the contents into the destination folder
            zip_ref.extractall(destination_folder)
        
        print(f"Extracted {filename} into {destination_folder}")
    else:
        print(f"{filename} is not a zip file. Skipping.")

print("Unzipping complete.")

# the unzip process isn't working as expected, we have encapsulated zip' files, so I need to do this again

## Sanity checks with the command line

In [ ]:
# using my trusty command line
#!unzip -o "zips/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223.zip" -d "zips/unzipped/"
# !unzip -o 'zips/unzipped/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223000000_0000000381390815.zip' -d "zips/unzipped/"
!unzip -o 'zips/unzipped/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223003000_0000000381392042.zip' -d "zips/unzipped/"


In [ ]:
# test zip file
import zipfile
import os

# Define the path to the zip file and the extraction directory
zip_path = 'zips/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223.zip'
extraction_path = 'zips/unzipped'  # Directory where files will be extracted

# Ensure the extraction directory exists
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extraction_path)
    print(f"Contents of the ZIP file extracted to {extraction_path}")

# List the contents of the extraction directory
extracted_files = os.listdir(extraction_path)
print("Extracted files:")
for file in extracted_files:
    print(file)

# If you expect only one CSV and it's directly in the ZIP, let's find it
csv_files = [file for file in extracted_files if file.endswith('.csv')]
if csv_files:
    print("CSV files found:")
    for csv_file in csv_files:
        print(csv_file)
else:
    print("No CSV files found directly in the ZIP file.")

# unzip round 2

In [30]:
import os
import zipfile

# Define the source and destination folder paths
source_folder = 'unzipped_files'
destination_folder = 'unzipped_files_round2'

# Check if the destination folder exists, if not, create it
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

i=0
# List all the files in the source folder
for filename in os.listdir(source_folder):
    # Construct the full file path
    file_path = os.path.join(source_folder, filename)
    
    # Check if the file is a ZIP file
    if zipfile.is_zipfile(file_path):
        print(f"Unzipping {filename}")
        
        # Open the ZIP file
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Extract all the contents into the destination folder
            zip_ref.extractall(destination_folder)
        
        print(f"Extracted {filename} into {destination_folder}")
        i+=1
        if i>=100: # I tried it for the first 100 zips in the second directory and seems to be working
            break
    else:
        print(f"{filename} is not a zip file. Skipping.")

print("Unzipping complete.")



Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20230721000000_0000000392554511.zip
Extracted PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20230721000000_0000000392554511.zip into unzipped_files_round2
Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20240102120000_0000000407398261.zip
Extracted PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20240102120000_0000000407398261.zip into unzipped_files_round2
Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230710070000_0000000391594683.zip
Extracted PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230710070000_0000000391594683.zip into unzipped_files_round2
Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20240219203000_0000000411546379.zip
Extracted PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20240219203000_0000000411546379.zip into unzipped_files_round2
Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20230701213000_0000000390930929.zip
Extracted PUBLIC_ROOFTOP_PV_ACTUAL_SATELLITE_20230701213000_0000000390930929.zip into unzipped_files_round2
Unzipping PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_

# Read as pandas and concat

# Appendix: debugging to get the zips downloaded correctly

In [ ]:
# interlude: 2 hours of debugging to get the links right

# import requests
# import os
# import hashlib

# def verify_file_checksum(file_path, original_checksum):
#     """Verify file checksum."""
#     sha256_hash = hashlib.sha256()
#     with open(file_path, "rb") as f:
#         # Read and update hash in chunks of 4K
#         for byte_block in iter(lambda: f.read(4096), b""):
#             sha256_hash.update(byte_block)
#     return sha256_hash.hexdigest() == original_checksum

# # Define the folder name where files will be saved
# folder_name = 'zips'

# # Check if the folder exists, if not, create it
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

# for file_url in zip_urls:
#     print(file_url)
#     response = requests.get(file_url, stream=True)  # Stream the download

#     if response.status_code == 200:
#         filename = file_url.split('/')[-1]  # Extracts the filename
#         file_path = os.path.join(folder_name, filename)

#         # Save the file in the specified folder
#         with open(file_path, 'wb') as file:
#             for chunk in response.iter_content(chunk_size=8192):
#                 file.write(chunk)

#         print(f"Downloaded {filename} into {folder_name}")

#         # Example checksum verification (assuming you have the original checksum)
#         # original_checksum = 'the_checksum_provided_for_the_file'
#         # if verify_file_checksum(file_path, original_checksum):
#         #     print(f"Checksum verified for {filename}")
#         # else:
#         #     print(f"Checksum mismatch for {filename}")
#     else:
#         print(f"Failed to download {filename}. Status code: {response.status_code}")

        
# import requests

# def download_file(url, filename):
#     """
#     Download a file from a URL and save it to the local filesystem.

#     Parameters:
#     - url: The URL of the file to download.
#     - filename: The local filename to save the downloaded file.
#     """
#     # Send a GET request to the URL
#     response = requests.get(url)
    
#     # Check if the request was successful
#     if response.status_code == 200:
#         # Open a local file with write-binary ('wb') mode
#         with open(filename, 'wb') as f:
#             f.write(response.content)
#         print(f"File downloaded successfully: {filename}")
#     else:
#         print(f"Failed to download file. HTTP Status Code: {response.status_code}")

# # Example usage
# url = zip_urls[0]
# filename = 'local_filename.zip'
# download_file(url, filename)

In [ ]:
url

In [ ]:
https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230223.zip